In [1]:
import tsunamibayes as tb
import numpy as np

# Testing Haversine (Distance Between Points on Globe)

In [2]:
R = 6.3781e6    #Radius of the earth
lat1 = np.array([50.06638889])    #Just two random lat/lon coordinates for testings purposes
lon1 = np.array([5.71472222])
lat2 = np.array([58.64388889])
lon2 = np.array([3.070000])
distance = tb.utils.haversine(lat1,lon1,lat2,lon2)
distance/10000      #This is the correct distance according to some other distance calculators found on Google.

In [3]:
import matplotlib.pyplot as plt
pointsx = [lat1, lat2]
pointsy = [lon1, lon2]
plt.plot(pointsx, pointsy, 'ro')
plt.plot(pointsx,pointsy)
plt.show()

# Creat a Reference Curve Fault Object

In [4]:
import pandas as pd     #Pandas is the library that contains some useful objects/functions for storing and organizing data.

df = pd.read_excel ('Flores.xlsx')     #df stands for data frame for the Flores fault.
dfwalinae = pd.read_excel('Walinae.xlsx')    #the data frame containing the coordinates for the Walinae fault. 
print(dfwalinae)
print (df)

In [5]:
latFlores = df.iloc[:,8]    #We slice here the important information: lat/lon/strike coordinates. We don't really use the other columns according to my knowledge
lonFlores = df.iloc[:,7]    
strikeFlores = df.iloc[:,9]
latWali = dfwalinae.iloc[:,8]
lonWali = dfwalinae.iloc[:,7]
strikeWali = dfwalinae.iloc[:,9]

#From what I understand, these slices are not numpy Arrays, they still contain the column title/index.


In [6]:
latFlores = latFlores.values   #We slice here the important information: lat/lon/strike coordinates. We don't really use the other columns according to my knowledge
lonFlores = lonFlores.values    
strikeFlores = strikeFlores.values
latWali = latWali.values
lonWali = lonWali.values
strikeWali = strikeWali.values

In [7]:
plt.plot(lonFlores, latFlores)
ax = plt.gca()
ax.set_aspect('equal')
plt.show()
#Flores fault lat/lon. 

In [8]:
plt.plot(lonWali, latWali)
ax = plt.gca()
ax.set_aspect('equal')
plt.show()
#Walinae fault lat/lon...
#This one will be a little weirder to make sure we don't have weird discontinuties when we interpolate the strike angle.

In [9]:
plt.plot(lonFlores, strikeFlores)
plt.show()

In [10]:
plt.plot(strikeWali, latWali)
plt.show()

In [11]:
#Creating the bounds for Flores fault object.
minlatF = latFlores.min()
maxlatF = latFlores.max()
minlonF = lonFlores.min()
maxlonF = lonFlores.max()

minlatW = latWali.min()
maxlatW = latWali.max()
minlonW = lonWali.min()
maxlonW = lonWali.max()


In [12]:
boundsFlores = {'lon_min' : minlonF,'lon_max' : maxlonF,
            'lat_min' : minlatF, 'lat_max':maxlatF}
boundsWali = {'lon_min' : minlonW,'lon_max' : maxlonW,
            'lat_min' : minlatW, 'lat_max':maxlatW}

In [13]:
#Using some test dip/depth curves. Essentially we assume that the dip/depth are constant throughout. 

depthcurve = lambda depth : 25*1000
dipcurve = lambda dip : 25

In [14]:
floresFault = tb.fault.ReferenceCurveFault(latFlores,lonFlores,strikeFlores,depthcurve,dipcurve,boundsFlores)
walinaeFault = tb.fault.ReferenceCurveFault(latWali, lonWali, strikeWali, depthcurve, dipcurve, boundsWali)
#Hey, it works! 

In [15]:
walinaeFault

# Test the Quad_Interp Function
This is most likely obsolete

In [16]:
x = np.array([1,2,3])
y = np.array([100,-3,25])

In [17]:
tb.fault.ReferenceCurveFault.quad_interp(x,y)

In [18]:
#Make sure to always use numPy arrays here!!!

x = np.array([1,2,3])
y = np.array([100,-3,25])
cof= tb.fault.ReferenceCurveFault.quad_interp(x,y)

graphx = np.linspace(1,3,100)
graphx = np.array(graphx)
graphy = cof[0]*graphx**2 + cof[1]*graphx + cof[2]
plt.plot(x,y,'ro')
plt.plot(graphx, graphy)
plt.show()

# Testing the Creation of Depth/Dip Curves

In [19]:
#Some debugging to do here...

xtest = np.array([-5,10,15])
ytest = np.array([20,25,30])
depthcurve, dipcurve = tb.fault.ReferenceCurveFault.depth_dip_curves(xtest,ytest,20)

In [20]:
xplot = np.linspace(0,20,100)
yplotdepth = depthcurve(xplot)
yplotdip = dipcurve(xplot)
plt.plot(xplot,yplotdepth,'b')
plt.plot(xplot,yplotdip,'r')
plt.show()

# Test Circ Mean (weighted mean of various angles)

In [21]:
#I simply gave each point a weight of 1 here.
weights = np.array([1,1,1])
strikeAngles = np.array([96, 90, 87])
tb.fault.ReferenceCurveFault.circmean(strikeAngles,weights)

# Testing the Side Method

### This function seems to handle arrays just fine!

In [22]:
plt.axvline(x=2)
plt.plot(1,1,'o')
plt.plot(3,1,'o')
plt.show()

In [23]:
tb.fault.ReferenceCurveFault.side(1,1,2,1,90)

In [24]:
tb.fault.ReferenceCurveFault.side(3,1,2,1,90)

In [25]:
closeLat = 2    #This is just a made up "closest point" to test whether we can pass in an array of points. 
closeLon =2 
latPoints = np.array([1, 1.5, 2.5, 3])
lonPoints = np.array([1,1.5,2.5,3])
plt.axvline(x=2)
plt.plot(latPoints, lonPoints, 'x')
plt.show()

In [26]:
tb.fault.ReferenceCurveFault.side(latPoints,lonPoints,closeLat, closeLon, 180)

In [27]:
testlat = np.array([-7.7])
testlon = np.array([115])

testlatArray = np.array([-7.7, -7.7])
testlonArray = np.array([115,118])
testlatArray[:, np.newaxis]

In [28]:
plt.plot(lonFlores, latFlores)
plt.plot(testlon, testlat, 'bx')
plt.show()

# Testing Distances Function

In [29]:
#This works well for one point, but we still need to figure out how the arrays will broadcast. 
testlatArray[:,np.newaxis]

In [30]:
[minDist, idxMin] = floresFault.distance(testlatArray[:,np.newaxis], testlonArray[:,np.newaxis], retclose= True)

In [31]:
tb.utils.haversine(testlatArray[:,np.newaxis], testlonArray[:,np.newaxis], floresFault.latpts[np.newaxis,:],floresFault.lonpts[np.newaxis,:])

In [32]:
# We need to find a way to use the min() function on each entry of the array. 

In [33]:
minDist, idxMin = floresFault.distance(testlat, testlon, retclose= True)

In [34]:
[minDist, idxMin]

In [35]:
plt.plot(lonFlores, latFlores)
plt.plot(testlon, testlat, 'bx')
plt.plot(lonFlores[idxMin], latFlores[idxMin], 'ro')
ax = plt.gca()
ax.set_aspect('equal')
plt.show()

In [36]:
floresFault.side(testlat, testlon, latFlores[idxMin], lonFlores[idxMin], strikeFlores[idxMin])

# Testing of Dip, Strike, and Depth Curves

In [37]:
floresFault.strike_map(testlat, testlon)

In [38]:
floresFault.strike_map(testlatArray[:,np.newaxis], testlonArray[:,np.newaxis])

In [39]:
floresFault.depth_map(testlat, testlon, retside = True)

In [40]:
floresFault.depth_map(testlatArray[:,np.newaxis], testlonArray[:,np.newaxis], retside = True)
#Intersting...why does this only return a single depth value...

In [41]:
floresFault.dip_map(testlat, testlon)

In [42]:
floresFault.dip_map(testlatArray[:,np.newaxis], testlonArray[:,np.newaxis])

# Testing for Broadcasting/np.newaxis

In [43]:
lat1 = np.array([0,1])
lon1 = np.array([10,20])

faultlats = np.array([0,1,2,3,4,5,6,7,8,9])
faultlons = np.array([0,10,20,30,40,50,60,70,80,90])


In [44]:
faultlats[np.newaxis, :]-lat1[:,np.newaxis]

In [45]:
lon1[:,np.newaxis]-faultlons[np.newaxis, :]

In [46]:
tb.utils.haversine(lat1[:,np.newaxis],lon1[:,np.newaxis], faultlats[np.newaxis, :],faultlons[np.newaxis, :])